In [1]:
import os
import re
import yaml
import json
import ast
import praw
from crewai import Agent, Task, Crew, Process
from langchain_openai import ChatOpenAI

from textwrap import dedent
from reddit_helper import *    

from langchain.llms import OpenAI, Ollama
from langchain_openai import ChatOpenAI

from pydantic import BaseModel
from datetime import datetime, timedelta

/opt/conda/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
api_file_path = 'api_keys.json'
with open(api_file_path, 'r') as file:
    api_keys = json.load(file)        
openai_gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])
openai_gpt4 = ChatOpenAI(model_name="gpt-4", 
                                temperature=0.7,
                                openai_api_key=api_keys['openai'])        
llama31_8b = Ollama(model='llama3.1:8b',)
gemma2_9b = Ollama(model='gemma2:9b',)

In [3]:
cfg_file_path = 'casaai_config.yaml'
agent_cfg_path = 'agents.yaml'
task_cfg_path = 'tasks.yaml'

with open(cfg_file_path, 'r') as yaml_file:
    cfg = yaml.safe_load(yaml_file)
product_long = cfg.get('product_long_description', '')      
product_short = cfg.get('product_short_description', '')
product_name = cfg.get('product_short_description', '')
domain = cfg.get('domain')
broad_keywords = cfg.get('broad_keywords', [])
output_format_1 = cfg.get('output_format_1')

with open(agent_cfg_path, 'r') as yaml_file:
    agent_cfg_data = yaml.safe_load(yaml_file)

with open(task_cfg_path, 'r') as yaml_file:
    task_cfg_data = yaml.safe_load(yaml_file)

In [4]:
tip_text = "If you do your BEST WORK, I'll give you a $100 commission!"

In [34]:
def re_decision_string(input_string):

    pattern = r"'comment_id':\s*'([^']*)',\s*'decision':\s*(\d+)"

    # Use re.findall to extract the values
    matches = re.findall(pattern, input_string)

    if matches:
        comment_id, decision = matches[0]
    else:
        comment_id = "na"
        decision = "discard"

    if "justification" in input_string:
        justification = input_string.split("justification")[-1]
    else:
        justification = "Not available"
    
    return comment_id, decision, justification

In [35]:
def re_score_string(input_string):

    pattern = r"'comment_id':\s*'([^']*)',\s*'score':\s*(\d+)"

    # Use re.findall to extract the values
    matches = re.findall(pattern, input_string)

    if matches:
        comment_id, score = matches[0]
    else:
        comment_id = "na"
        score = 0

    if "justification" in input_string:
        justification = input_string.split("justification")[-1]
    else:
        justification = "Not available"
    
    return comment_id, score, justification

In [59]:
def get_calendar_date(unix_timestamp):
    date_obj = datetime.utcfromtimestamp(unix_timestamp)
    readable_date = date_obj.strftime('%Y-%m-%d %H:%M:%S')
    return readable_date

In [78]:
def prep_decision_output(decision_lst, old_comments):
    decision_analysis_lst = []
    decision_bad_list = []
    tot_comments = 0
    bad_cnt = 0
    decision_relevant_json_list = []
    decision_discarded_json_list = []
    for item in decision_lst:  
        sep = "^"
        for comment in item:
            tot_comments += 1
            try:
                comm_key = list(comment.keys())[0]
                comm_id  = comment[comm_key].strip()
                if comm_id in comment_dict_list:                 
                    parent_id = comment_dict[comm_id]["parent_id"]
                    if parent_id is None:
                        parent_id = 'NA'
                    decision = comment['decision']
                    justify  = comment["justification"]
                    comment_text = comment_dict[comm_id]["text"]
                    text_str = sep + comm_id + sep + parent_id + sep + decision + sep + justify + sep + comment_text + sep
                    if comment['decision'] == 'relevant':
                        decision_relevant_json_list.append(comment)
                    else:
                        decision_discarded_json_list.append(comment)
                    decision_analysis_lst.append(text_str)            
                else: 
                    print(f'Comment ID not found : {comm_id}')
                    bad_cnt += 1
                    decision_bad_list.append(comm_id)
            except:
                print(f'comment key error : {comment}')
                bad_cnt += 1
    print(f'relevant : {len(decision_relevant_json_list)} discarded : {len(decision_discarded_json_list)} bad_cnt: {bad_cnt} total: {tot_comments}')
    
    old_comment_list = []
    for comment in old_comments:  
        sep = "^"
        comm_id  = comment["comment_id"]
        parent_id = comment["parent_id"]
        comment_text  = comment["text"]
        created_date = get_calendar_date(comment['created_utc'])
        age = str(comment["age"])
        text_str = sep + comm_id + sep + parent_id + sep + age + sep + created_date + sep + comment_text + sep
        old_comment_list.append(text_str)      
    print(f'Old comments already filtered out : {len(old_comments)}')
    
    output = (decision_relevant_json_list, decision_discarded_json_list, decision_analysis_lst, old_comment_list)
    return output

In [89]:
def prep_score_output(score_result_lst):
    analysis_lst = []
    bad_list = []
    tot_comments = 0
    bad_cnt = 0
    good_json_list = []
    sep = "^"
    for comment in score_result_lst:
        tot_comments += 1
        try:
            comm_key = list(comment.keys())[0]
            comm_id  = comment[comm_key].strip()
            
            if comm_id in comment_dict_list:  
                parent_id = comment_dict[comm_id]["parent_id"]
                if parent_id is None:
                    parent_id = 'NA'
                score = comment['score']
                age = comment_dict[comm_id]["age"]
                created = comment_dict[comm_id]["created"]
                justify  = comment["justification"]
                comment_text = comment_dict[comm_id]["text"]
                text_str = sep + comm_id + sep + parent_id + sep + str(score) + sep + str(age) + sep + created + sep + justify \
                            + sep + comment_text + sep
                analysis_lst.append(text_str) 
                good_json_list.append(comm_id)
            else: 
                print(f'Comment ID not found : {comm_id}')
                bad_cnt += 1
                bad_list.append(comm_id)
        except:
            print(f'comment key error : {comment}')
            bad_cnt += 1
    print(f'relevant : {len(good_json_list)} bad_cnt: {bad_cnt} total: {tot_comments}')
    output = (good_json_list, bad_list, analysis_lst)
    return output

In [36]:
backstory = agent_cfg_data['content_filter_analyst']['backstory'] 
goal = agent_cfg_data['content_filter_analyst']['goal']
role = agent_cfg_data['content_filter_analyst']['role'] 
content_filter_analyst = Agent(
                            role=role,
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=False,
                            llm=gemma2_9b,
                            )

In [37]:
class DecisionOutput(BaseModel):
    comment_id: str
    decision: str
    justification: str

In [38]:
description = task_cfg_data['content_filter_task']['description']
expected_out = task_cfg_data['content_filter_task']['expected_out'] 
        
content_filter_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=DecisionOutput,
                            agent=content_filter_analyst,
                            )

In [39]:
response_creation_crew = Crew(
    agents=[content_filter_analyst,],
    tasks=[content_filter_task,],
    verbose=False,
)

2024-09-01 08:13:45,936 - 140270313555776 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [11]:
reddit_posts, reddit_post_ids = fetch_reddit_test()
condensed_reddit_data, unique_post_ids, unique_comment_ids = condense_data(reddit_posts, reddit_post_ids)

post_cnt: 0 + 19 = 19
comm_cnt: 0 + 692 = 692
cond_cnt: 711 = 19 + 692
Cross_ck: 19 = 19


In [88]:
comment_dict = {}
tot_data = 0
for idx1, item in enumerate(condensed_reddit_data):
    for idx2, data in enumerate(item):
        tot_data += 1
        c_id = data['comment_id']
        p_id = data['parent_id']
        text = data['text']
        created_date = get_calendar_date(data['created_utc'])
        age = data['age']
        comment_dict[c_id] = {'comment_id': c_id, 'parent_id': p_id, 'text': text, 'age': age, 'created': created_date}
    print(idx1, ':', idx2)
comment_dict_list = list(comment_dict.keys())
print(len(comment_dict), '=', tot_data, len(comment_dict_list))

0 : 169
1 : 108
2 : 31
3 : 57
4 : 0
5 : 33
6 : 60
7 : 67
8 : 1
9 : 34
10 : 0
11 : 1
12 : 79
13 : 0
14 : 7
15 : 2
16 : 0
17 : 9
18 : 34
711 = 711 711


In [19]:
def get_data_details(comm_id):
    done = 0
    data_details = []
    while done==0:
        if comm_id in comment_dict:
            details = comment_dict[comm_id]
            data_details.append(details)
            comm_id = details['parent_id']
        else:
            done = 1
    return data_details

In [ ]:
decision_lst = []
old_comments = []
age_limit = 3
for idx, reddit_data_item in enumerate(condensed_reddit_data):
    comment_lst = []
    for idx2, comment_data in enumerate(reddit_data_item):
        if comment_data['age'] < age_limit:
            comm_id = comment_data['comment_id']  
            comment_text = comment_data['text']
            print(f'STARTING {idx} - {idx2} - {comm_id}')   
            input_dict = {"comment_id": comm_id,
                          "input_data": comment_text,
                          "product_long": product_long,
                          "product_short": product_short,
                          "domain": domain,
                          "broad_keywords": broad_keywords,
                          "output_format":output_format_1, 
                          "tip_text":tip_text}
            decision_result = response_creation_crew.kickoff(inputs=input_dict)
            try:
                json_out = ast.literal_eval(decision_result.json)
            except:
                print('Exception in json - trying re')
                _, d, j = re_decision_string(decision_result.raw)            
                json_out = {'comm_id': comm_id, 'decision': str(d), 'justification': str(j)}
            comment_lst.append(json_out)
        else:
            old_comments.append(comment_data)
    decision_lst.append(comment_lst)

In [66]:
decision_out = prep_decision_list(decision_lst, old_comments)

comment key error : {'ku3v9wy': {'decision': 'discard', 'justification': 'Short message, less than 15 words.'}}
comment key error : {'ku5l2mj': {'decision': 'relevant', 'justification': 'Seeks advice on skylight heat and shading.'}}
Comment ID not found : kq75xk
Comment ID not found : krct1qh
Comment ID not found : krclyt
comment key error : {'kcnf0d0': {'decision': 'discard', 'justification': 'Short message, less than 15 words.'}}
comment key error : {'k34z5z8': {'decision': 'discard', 'justification': 'Very short message, less than 7 words.'}}
Comment ID not found : iyawpm
Comment ID not found : 1
Comment ID not found : iyv709
relevant : 71 discarded : 117 bad_cnt: 10 total: 198
Old comments already filtered out : 513


In [67]:
for item in decision_out:
    print(len(item))
content_filter_relevant, content_filter_discard, content_filter_analysis, content_filter_old = decision_out
content_filter_relevant[0]

71
117
188
513


{'comment_id': 'ku4kjo6',
 'decision': 'relevant',
 'justification': "The comment expresses a desire for practical kitchen design that is both beautiful and functional, aligning with CasaAI's scope."}

In [69]:
with open('decision_result_v6_0901B.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in content_filter_analysis:
        file.write(item + "\n")  # Add a newline character after each string
with open('decision_old_v6_0901B.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in content_filter_old:
        file.write(item + "\n")  # Add a newline character after each string

print(f"Files has been written")

with open('decision_result_v6_0901B.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(f"Files Reloaded")

Files has been written
Files Reloaded


In [70]:
backstory = agent_cfg_data['content_scoring_analyst']['backstory'] 
goal = agent_cfg_data['content_scoring_analyst']['goal']
role = agent_cfg_data['content_scoring_analyst']['role'] 
content_scoring_analyst = Agent(
                            role=role,
                            goal=goal,
                            backstory=backstory,
                            allow_delegation=False,
                            verbose=False,
                            llm=gemma2_9b,
                            )

In [71]:
class ScoreOutput(BaseModel):
    comment_id: str
    score: float
    justification: str

In [72]:
description = task_cfg_data['content_scoring_task']['description']
expected_out = task_cfg_data['content_scoring_task']['expected_out'] 
        
content_scoring_task = Task(
                            description=description,
                            expected_output=expected_out,
                            output_json=ScoreOutput,
                            agent=content_filter_analyst,
                            )

In [73]:
score_creation_crew = Crew(
    agents=[content_scoring_analyst,],
    tasks=[content_scoring_task,],
    verbose=False,
)

2024-09-01 11:00:31,862 - 140270313555776 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [ ]:
score_result_lst = []
for idx, comment_data in enumerate(content_filter_relevant):     
    comm_id = comment_data['comment_id']        
    print(f'STARTING {idx} - {comm_id}')
    data_details = get_data_details(comm_id)      
    input_dict = {"comment_id": comm_id,
                  "input_data": data_details,
                  "product_long": product_long,
                  "product_short": product_short,
                  "domain": domain,
                  "product_name": product_name,
                  "tip_text":tip_text}
    scoring_result = score_creation_crew.kickoff(inputs=input_dict)
    try:
        json_out = ast.literal_eval(scoring_result.json)
    except:
        print('Exception in json - trying re')
        _, s, j = re_score_string(scoring_result.raw)            
        json_out = {'comm_id': comm_id, 'score': float(s), 'justification': str(j)}            
    score_result_lst.append(json_out)

In [76]:
score_result_lst[0], len(score_result_lst), len(relevant_json_list)

({'comment_id': 'ku4kjo6',
  'score': 8.2,
  'justification': "Expresses desire for a usable kitchen, aligns with CasaAI's functionality."},
 71,
 71)

In [77]:
thresh_list = [7.0, 7.5, 8.0, 8.5, 9.0, 9.5]

for thresh in thresh_list:
    thresh_up = 0
    for item in score_result_lst:
        if item['score'] >= thresh:
            thresh_up += 1
    print(thresh, ':', thresh_up, '/', len(score_result_lst))

7.0 : 33 / 71
7.5 : 28 / 71
8.0 : 17 / 71
8.5 : 12 / 71
9.0 : 3 / 71
9.5 : 0 / 71


In [90]:
score_out = prep_score_output(score_result_lst)
content_score_good, content_score_bad, content_score_analysis = score_out
print(len(content_score_good), len(content_score_analysis))

relevant : 71 bad_cnt: 0 total: 71
71 71


In [91]:
with open('score_result_v6_0901A.txt', "w") as file:
    # Iterate through the list and write each string to the file
    for item in content_score_analysis:
        file.write(item + "\n")  # Add a newline character after each string

print(f"File has been written")

with open('score_result_v6_0901A.txt', "r") as file:
    # Read all lines and strip the newline character from each line
    analysis_lst_loaded_back = [line.strip() for line in file.readlines()]
print(len(analysis_lst_loaded_back))

File has been written
111


In [96]:
# For each post-comment combo select the post-comments whose score >= 7
filter_thresh = 7.0
filtered_comment_ids = []
for item in score_result_lst:
    if item['score'] >= filter_thresh:
        filtered_comment_ids.append(item['comment_id'])
print(f'len(filtered_comment_ids) : {len(filtered_comment_ids)}')

len(filtered_comment_ids) : 33


In [ ]:
# Build response for these comments With Manager LLM + only 1 response writer + meta reviewer

In [98]:
response_writer = Agent(
          role="Response writer",
          goal="Write the response to the comments in a casual way including marketing message of the {product} ",
          backstory="You will be given json data of comments. These comments are coming from content filterer agent and"
                    "is relevant to the product that we are marketing. Your job is to draft a casual reply to the comment "
                    " which will sound genuine and inline with the general thread of discussion. You should also "
                    " gently insert a short (within 25 words) marketing message of {product} in between in a natural way.",
          allow_delegation=True,
          verbose=True,
          llm=gemma2_9b
)

In [99]:
meta_reviewer = Agent(
	role="Meta Reviewer",
	goal="You review the response crafted and provide suggestions to improve",
	backstory=(
    "You will receive the reply written by response writer agent. These replies are intended to market our {product} in the"
    "guise of providing genuine suggestions/answers to the original comment."
    "Your goal is to review these replies to ensure that "
    "message is interesting and marketing of product happens naturally while catching attention."
	),
  allow_delegation=True,
  verbose=True,
  llm=gemma2_9b
)

In [100]:
response_write_task = Task(
    description=(
        "1. Comments you are getting as input are suggestions/advises to the help seeked in title or may be follow-up questions to understand the ask better.\n"
        "2. Read the comments from input and understand whether it is a suggestion/advise/follow-up question.\n"
        "3. Once done, draft text based on the type of comment.\n"
        "4. Reply should sound genuine and in-synch with the overall thread of discussion.\n"
        "5. Response style must be crisp, casual, friendly and must not include any reference to the product.\n"
        "6. Please include keywords mentioned in the original comment in your response text.\n"
        "7. Please be careful not to include keywords that were NOT mentioned in the original comment in your response text.\n"
        "8. Insert marketing text for the {product} in between these casual replies.\n"
        "9. Marketing text inserted must feel natural and in-synch and must not exceed 25 words.\n"
        "10. If needed, inlcude the link to the product website.\n"
        "11. Input data to be used is {input_data}.\n"
        "12. Response you drafted will be sent to meta reviewer for critic review.\n"
        "13. You have to refine the response based on review comments from meta reviewer and create the output.\n"
        "14. For output, include the 'Comment_id' and 'text' that you modified after review.\n"
        "15. You are the one responsible for providing final output after taking care of review comments.\n"
        "16. Please ensure that final output you are providing has reply for each comment provided in input_data.\n"
    ),
    expected_output="A json format of 'Comment_id':'response-text'",
    agent=response_writer,
)

In [102]:
meta_review_task = Task(
    description=(
        "1. You are getting the casual replies drafted by response writer agent.\n"
        "2. These casual replies also has marketing text inserted for the {product} in between.\n"
        "3. Review these casual replies considering below points:\n"
            "a. optimize content for search engines, ensuring that it ranks well and attracts organic traffic.\n"
            "b. marketing message is catchy and gets attention.\n"
            "c. There are no offensive words or content in the message.\n"
        "4. Make sure your suggestion is concise (within 3 bullet points), concrete and to the point.\n"
        "5. For output, include the 'Comment_id' and 'review_comment' .\n"
        "6. Your review comments will be taken care by response writer agent. \n"
    ),
    agent=meta_reviewer,
    expected_output="A json format of 'Comment_id':'review_comment'",
)

In [103]:
manager_llm = Ollama(model='gemma2:9b',)

In [104]:
response_creation_crew = Crew(
    agents=[response_writer, meta_reviewer],
    tasks=[response_write_task, meta_review_task, response_write_task],
    verbose=True,
    manager_llm=manager_llm,
    process=Process.hierarchical
)

2024-09-01 12:16:13,111 - 140270313555776 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


In [105]:
# result = response_creation_crew.kickoff(inputs={"product": casa_ai_descr, "input_data": comments_lst})